In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pathlib
from sklearn.metrics import precision_recall_fscore_support
import torch
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from ray import tune
sns.set(rc={'figure.figsize':(9, 7)})

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# Coral Ordinal Function and Custom metric defining here
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
MODEL_PATH = '../data/models/'
DATA_DIR = '../data/'


2023-12-07 16:54:20.179774: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 16:54:20.179837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 16:54:20.180716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 16:54:20.187948: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 16:54:21.086747: W tensorflow/compiler/tf2

In [2]:
# Not to use in ipynb file
# import argparse
# # Create the parser
# parser = argparse.ArgumentParser(description="Script for generating the deep ensemble hyperparameter results")
# # Add the --value argument
# parser.add_argument('--dv', type=int, help="Data version")
# # Parse the arguments
# args = parser.parse_args()
# # Asking the user for their name
# data_type = int(args.dv)

data_type = 4

In [3]:

version = 'stl_1'
log_version = 'stl_1'
EPOCHS = 200

DATA_DIR = '../data'

pd.set_option('display.max_columns', None)

In [4]:
data_version = ''
if data_type != 4:

    data_version = 'synthetic'

    path = f'../data/new_data/data{data_type}/'

    train_df = pd.read_csv(f'{path}/Data{data_type}_Train.csv')
    val_df = pd.read_csv(f'{path}/Data{data_type}_Val.csv')
    test_df = pd.read_csv(f'{path}/Data{data_type}_Test.csv')

    tensor_train_data = torch.Tensor(train_df.Input1).unsqueeze(1)
    tensor_train_label = torch.Tensor(train_df.Output)

    tensor_val_data = torch.Tensor(val_df.Input1).unsqueeze(1)
    tensor_val_label = torch.Tensor(val_df.Output)

    tensor_test_data = torch.Tensor(test_df.Input1).unsqueeze(1)
    tensor_test_label = torch.Tensor(test_df.Output)

    train_dataset = TensorDataset(tensor_train_data, tensor_train_label)
    val_dataset = TensorDataset(tensor_val_data, tensor_val_label)

else:
    data_version = 'household_power'
    path = f'../data/{data_version}/'

    # feature_x = pd.read_csv(f'{path}/feature_x.csv')
    train_x = np.load(f'{path}/train_x.npy')
    train_y = np.load(f'{path}/train_y.npy')

    val_x = np.load(f'{path}/val_x.npy')
    val_y = np.load(f'{path}/val_y.npy')

    test_x = np.load(f'{path}/test_x.npy')
    test_y = np.load(f'{path}/test_y.npy')

    tensor_train_data = torch.Tensor(train_x)
    tensor_train_label = torch.Tensor(train_y)

    tensor_val_data = torch.Tensor(val_x)
    tensor_val_label = torch.Tensor(val_y)

    tensor_test_data = torch.Tensor(test_x)
    tensor_test_label = torch.Tensor(test_y)

    train_dataset = TensorDataset(tensor_train_data, tensor_train_label)
    val_dataset = TensorDataset(tensor_val_data, tensor_val_label)

In [5]:
models = {}

In [6]:
class DeepEnsembleNet(nn.Module):
    def __init__(self, num_features, hidden_size, n_layers):
        super(DeepEnsembleNet, self).__init__()

        layers = []
        input_size = num_features

        # Create 'n' hidden layers
        for _ in range(n_layers):
            layers.append(nn.Linear(input_size, hidden_size))
            input_size = hidden_size  # The output of each layer is the input to the next

        self.layers = nn.ModuleList(layers)
        self.mu = nn.Linear(hidden_size, 1)
        self.var = nn.Linear(hidden_size, 1)

    def forward(self, x):
        for layer in self.layers:
            x = torch.relu(layer(x))
        mu = self.mu(x)
        var = torch.exp(self.var(x))
        return mu, var
    

In [7]:
def train_de(config):

    hidden_size = int(config['hidden_size'])
    num_layers = int(config['num_layers'])
    patience = int(config['patience'])
    batch_size = int(config['batch_size'])
    num_models = int(config['num_models'])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    model_name = f'hs_{hidden_size}_nl_{num_layers}_do_{config["num_models"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}.pt'

    models[f'{model_name}']= DeepEnsembleNet(config["num_features"], hidden_size, num_layers)

    device = 'cpu'
    models[f'{model_name}'].to(device)
    
    best_model_dir = f'./models/'
    pathlib.Path(best_model_dir).mkdir(parents=True, exist_ok=True) 
    best_model_path = f'{best_model_dir}/hs_{hidden_size}_nl_{num_layers}_do_{config["num_models"]}_ep_{config["epochs"]}_p_{config["patience"]}_lr_{config["lr"]}_bs_{batch_size}'
    best_val_loss = float('inf')
    epochs_no_improve = 0

    # Construct ensemble
    deep_ensemble = [DeepEnsembleNet(config["num_features"], hidden_size, num_layers).to(device) for i in range(num_models)]
    criterion = torch.nn.GaussianNLLLoss(eps=1e-02)
    optimizers = [optim.Adam(m.parameters(), lr=config["lr"]) for m in deep_ensemble]

    for epoch in range(config['epochs']):
        # Train loop
        for batch in train_loader:
            x = batch[0].to(device)
            y = batch[1].to(device)

            losses = []
            mus = []
            vars = []
            for i, model in enumerate(deep_ensemble):
                optimizers[i].zero_grad()
                mu, var = model(x)
                loss = criterion(mu, y, var)
                loss.backward()
                optimizers[i].step()

                losses.append(loss.item())
                mus.append(mu)
                vars.append(var)
            loss = sum(losses)/len(losses)

        # Validation loop to monitor early stopping
        model.eval()  # Set the model to evaluation mode
        val_losses = []
        for batch in val_loader:  # Assuming you have a validation data loader
            x = batch[0].to(device)
            y = batch[1].to(device)

            with torch.no_grad():  # No gradient calculation for validation data
                for model in deep_ensemble:
                    mu, var = model(x)
                    val_loss = criterion(mu, y, var)
                    val_losses.append(val_loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping triggered at epoch {epoch}')
                break

        print(f'Epoch: {epoch}|Train Loss: {loss}|Val Loss: {avg_val_loss}')
    
    tune.report(train_loss=loss,best_val_loss=best_val_loss)


In [8]:


def main(num_samples=100, max_num_epochs=500):

    if data_type != 4:
        num_features = 1
    else:
        num_features = 7

    ray.init()

    config = {
        "num_features": num_features,
        "hidden_size": tune.quniform(1, 200, 1),
        "num_layers": tune.quniform(2, 10, 1),
        "num_models": 5,
        "lr": tune.loguniform(1e-4, 1e-1),
        "epochs": max_num_epochs,
        "patience": tune.quniform(1, 50, 1),
        "batch_size": tune.quniform(1, 20, 1)
    }

    from ray.tune.search import ConcurrencyLimiter
    from ray.tune.search.bayesopt import BayesOptSearch
    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
    algo = ConcurrencyLimiter(algo, max_concurrent=25)

    result = tune.run(
            tune.with_parameters(train_de),
            config=config,
            resources_per_trial={"cpu": 1},
            metric="best_val_loss",
            mode="min",
            search_alg=algo,
            num_samples=num_samples,
            # max_concurrent_trials=20,
            local_dir= f'/home/sgupta/WORK/Triplevel_transformer_model/baselines/hyperparameter_search',
            name=f"experiment_de_{data_type}",
            max_failures=7,
            raise_on_failed_trial=False
        )

    temp = result.dataframe()   
    hyperparams_path = f'./hyperparameter_search/'
    temp.to_csv(f"{hyperparams_path}/RAY_RESULTS_de_data{data_type}.csv")


In [9]:
import ray

main(500,500)
#  Stop Ray
ray.shutdown()

2023-12-07 16:54:27,205	INFO worker.py:1621 -- Started a local Ray instance.
2023-12-07 16:54:28,821	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-07 16:54:28,841	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-12-07 16:54:28,842	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-12-07 16:54:28,842	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2023-12-07 16:54:28,843	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2023-12-07 16:54:28,843	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform

(train_de pid=9777) /media/seconddrive/sgupta/anaconda3/envs/py39SAM/lib/python3.9/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
(train_de pid=9777)   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
(raylet) [2023-12-07 16:54:36,189 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42478104576; capacity: 1006450962432. Object creation will fail if spilling is required.
(train_de pid=9889) /media/seconddrive/sgupta/anaconda3/envs/py39SAM/lib/

(train_de pid=9839) Epoch: 0|Train Loss: -2.2496002197265623|Val Loss: -1.8243953763594911


(raylet) [2023-12-07 16:56:56,346 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42471292928; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:57:06,357 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42470817792; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:57:16,368 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42470363136; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 16:57:26,380 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42469916672; capacity: 1006450962432. Object creation will fa

(train_de pid=9889) Epoch: 0|Train Loss: -0.5279855728149414|Val Loss: -1.3783716596492894


...

(raylet) [2023-12-07 17:00:06,539 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42502537216; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 17:00:16,549 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42502094848; capacity: 1006450962432. Object creation will fail if spilling is required.


(train_de pid=10332) Epoch: 0|Train Loss: -1.855073881149292|Val Loss: -1.5001358865575702


(raylet) [2023-12-07 17:00:26,560 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42501656576; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 17:00:36,571 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42501103616; capacity: 1006450962432. Object creation will fail if spilling is required.


(train_de pid=9889) Epoch: 1|Train Loss: -1.7334696054458618|Val Loss: -1.451615428046821
(train_de pid=10238) Epoch: 0|Train Loss: -1.6186848163604737|Val Loss: -1.3958876033773706


(raylet) [2023-12-07 17:00:46,582 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42500661248; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 17:00:56,593 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42500231168; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 17:01:06,604 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42499674112; capacity: 1006450962432. Object creation will fail if spilling is required.
(raylet) [2023-12-07 17:01:16,616 E 9002 9022] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-07_16-54-24_461926_8758 is over 95% full, available space: 42499219456; capacity: 1006450962432. Object creation will fa